In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc
import requests
from urllib.parse import urlparse

In [2]:
basedir = './국토교통부_실거래가_공개시스템/토지/'
filenames = [f for f in os.listdir(basedir) if ('토지' in f) & ('.csv' in f)]
filenames

['토지(매매)__실거래가_20060101_20061231.csv',
 '토지(매매)__실거래가_20070101_20071231.csv',
 '토지(매매)__실거래가_20080101_20081231.csv',
 '토지(매매)__실거래가_20090101_20091231.csv',
 '토지(매매)__실거래가_20100101_20101231.csv',
 '토지(매매)__실거래가_20110101_20111231.csv',
 '토지(매매)__실거래가_20120101_20121231.csv',
 '토지(매매)__실거래가_20130101_20131231.csv',
 '토지(매매)__실거래가_20140101_20141231.csv',
 '토지(매매)__실거래가_20150101_20151231.csv',
 '토지(매매)__실거래가_20160101_20161231.csv',
 '토지(매매)__실거래가_20170101_20171231.csv',
 '토지(매매)__실거래가_20180101_20181231.csv',
 '토지(매매)__실거래가_20190101_20191231.csv',
 '토지(매매)__실거래가_20200101_20201231.csv',
 '토지(매매)__실거래가_20210101_20210331.csv']

In [8]:
for f in filenames:
    df = pd.read_csv(basedir + f, encoding='euc-kr', header=14)
    display(df.head())
    year = str(df['계약년월'].iloc[0])[:4]
    print(year)
    df.to_csv(basedir + '토지(매매)__실거래가_{}0101_{}1231.csv'.format(year, year), index=False)

,시군구,번지,지목,용도지역,도로조건,계약년월,계약일,계약면적(㎡),거래금액(만원),지분구분
0,서울특별시 구로구 오류동,2*,대,제2종일반주거지역,8m미만,200601,1,23.14,"1,890",
1,서울특별시 관악구 신림동,산9*,임야,제3종일반주거지역,25m미만,200601,2,26.00,"1,432",지분
2,서울특별시 강남구 역삼동,7**,대,제3종일반주거지역,8m미만,200601,2,393.30,"267,750",
3,서울특별시 서초구 신원동,3**,전,개발제한구역,8m미만,200601,2,684.00,"53,820",
4,서울특별시 관악구 신림동,산1**,임야,제3종일반주거지역,25m미만,200601,3,24.00,"1,325",지분


2006


,시군구,번지,지목,용도지역,도로조건,계약년월,계약일,계약면적(㎡),거래금액(만원),지분구분
0,서울특별시 강북구 미아동,7**,대,제2종일반주거지역,8m미만,200701,2,82.00,"10,900",
1,서울특별시 강북구 미아동,7**,도로,제2종일반주거지역,8m미만,200701,2,2.00,100,
2,서울특별시 용산구 원효로4가,8*,대,제2종일반주거지역,8m미만,200701,2,433.80,"165,000",
3,서울특별시 동대문구 용두동,6**,대,준주거지역,12m미만,200701,2,11.00,"2,000",지분
4,서울특별시 노원구 중계동,1*,대,개발제한구역,8m미만,200701,2,99.18,"24,000",


2007


,시군구,번지,지목,용도지역,도로조건,계약년월,계약일,계약면적(㎡),거래금액(만원),지분구분
0,서울특별시 서대문구 홍은동,산*,임야,개발제한구역,-,200801,1,1521.00,"1,000",
1,서울특별시 용산구 이태원동,5*,대,준주거지역,8m미만,200801,2,2.67,"1,050",지분
2,서울특별시 종로구 신문로2가,1**,대,일반상업지역,-,200801,2,4.00,"1,311",
3,서울특별시 용산구 이태원동,6*,대,준주거지역,25m이상,200801,2,1.67,"1,500",지분
4,서울특별시 종로구 신문로2가,9*,대,일반상업지역,8m미만,200801,2,12.20,"11,220",


...

2018


,시군구,번지,지목,용도지역,도로조건,계약년월,계약일,계약면적(㎡),거래금액(만원),지분구분
0,서울특별시 성북구 정릉동,8**,잡종지,개발제한구역,8m미만,201901,2,8.45,1,지분
1,서울특별시 성동구 옥수동,3**,대,제2종일반주거지역,8m미만,201901,2,2.48,"1,000",지분
2,서울특별시 강남구 삼성동,9*,대,제3종일반주거지역,25m미만,201901,2,663.30,"1,060,000",
3,서울특별시 동작구 신대방동,3**,대,제1종일반주거지역,-,201901,2,49.00,"1,350",지분
4,서울특별시 금천구 가산동,5**,대,준공업지역,-,201901,2,2.00,"1,555",지분


2019


,시군구,번지,지목,용도지역,도로조건,계약년월,계약일,계약면적(㎡),거래금액(만원),지분구분
0,서울특별시 동작구 상도동,1**,잡종지,제2종일반주거지역,8m미만,202001,2,4.67,"1,400",지분
1,서울특별시 도봉구 도봉동,산5*,임야,개발제한구역,8m미만,202001,2,429.75,"1,677",지분
2,서울특별시 마포구 염리동,8*,대,제2종일반주거지역,8m미만,202001,2,20.00,"10,000",
3,서울특별시 성동구 성수동1가,6**,대,준주거지역,-,202001,2,43.00,"104,050",
4,서울특별시 성동구 성수동1가,6**,대,준주거지역,-,202001,2,43.00,"104,050",


2020


In [4]:
for f in filenames:
    year = f.replace('.csv', '')[-8:-4]
    if (year == '2020') | (year == '2021'):
        df = pd.read_csv(basedir + f, encoding='euc-kr', header=14)
        display(df.head())
        year = str(df['계약년월'].iloc[0])[:4]
        print(year)
        df.to_csv(basedir + '토지(매매)__실거래가_{}0101_{}1231.csv'.format(year, year), index=False)

,시군구,번지,지목,용도지역,도로조건,계약년월,계약일,계약면적(㎡),거래금액(만원),지분구분,해제사유발생일
0,서울특별시 동작구 상도동,1**,잡종지,제2종일반주거지역,8m미만,202001,2,4.67,"1,400",지분,NaN
1,서울특별시 도봉구 도봉동,산5*,임야,개발제한구역,8m미만,202001,2,429.75,"1,677",지분,NaN
2,서울특별시 마포구 염리동,8*,대,제2종일반주거지역,8m미만,202001,2,20.00,"10,000",,NaN
3,서울특별시 성동구 성수동1가,6**,대,준주거지역,-,202001,2,43.00,"104,050",,NaN
4,서울특별시 성동구 성수동1가,6**,대,준주거지역,-,202001,2,43.00,"104,050",,NaN


2020


,시군구,번지,지목,용도지역,도로조건,계약년월,계약일,계약면적(㎡),거래금액(만원),지분구분,해제사유발생일
0,서울특별시 동작구 노량진동,2**,도로,제3종일반주거지역,-,202101,2,50.00,"50,000",,NaN
1,서울특별시 동작구 노량진동,3**,대,제3종일반주거지역,-,202101,2,74.00,"58,000",지분,NaN
2,서울특별시 도봉구 도봉동,4**,잡종지,제1종일반주거지역,8m미만,202101,3,34.00,"15,427",지분,NaN
3,서울특별시 관악구 신림동,산6*,임야,자연녹지지역,-,202101,3,3206.60,"4,000",지분,NaN
4,서울특별시 도봉구 도봉동,4**,잡종지,제1종일반주거지역,8m미만,202101,3,11.33,"5,100",지분,NaN


2021
